In [ ]:
import sys
import yaml
sys.path.append('/home/harmon/data/Codes/FJ-codes/DisperNetLocal/')
import dispernet_local_latest as dispernet
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
proj_name = proj['name']
proj_name = 'output_FJSJ_09-10/'

In [ ]:
filename = proj_name+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = proj_name+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_ds = info_basic['dir_ds']
key_ds = info_basic['key_subworks']
#key_ds = ['09-04']

# create workspace for Dispernet
dir_inv = proj_name+'inv_dispernet/'
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
info_basic['dir_inv_dispernet'] = dir_inv

with open(proj_name+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [ ]:
workPath = dir_inv
filePath = workPath + 'h5/'
curveFilePath = workPath + 'data_period/'
threshold = 0.1
freqSeries = info_basic_bi['f']

In [ ]:
workPath

In [ ]:
f = info_basic_bi['f']
# 提取f小于0.5并且大于0.1的频率
f = f[(f>0.1) & (f<0.5)]

In [ ]:
f

In [ ]:
curveFilePath

In [ ]:
def get_file_list(basis_dir="./", begin="", end=""):
		path_list = os.listdir(basis_dir)
		list_final = []
		for partial in path_list:
			if begin and end:
				if partial[:len(begin)] == begin and partial[-len(end):] == end:
					list_final.append(partial)
					
			elif end:
				if partial[-len(end):] == end and partial[0] != '.':
					list_final.append(partial)
			
			elif begin:
				if partial[:len(begin)] == begin:
					list_final.append(partial)
					
			else:
				list_final.append(partial)
				
		return list_final
		
def curve_save(curve, fileName, curveFilePath='./'):
		if curve == [] or np.array(curve).ndim < 2:
			print('No curves to save yet.')
		else:
			if len(curve) > 1:
				if curve.shape[1] !=2 and  curve.shape[1] !=4:
					curve = curve[np.argsort(curve[:,-1])]
					for mode in range(int(max(curve[:,-1])+1)):
						curveInMode = curve[curve[:,-1] == mode]					
						curve[curve[:,-1] == mode] = curveInMode[np.argsort(curveInMode[:,0])]
			
			if curve.shape[1] > 2:		
				np.savetxt(curveFilePath + fileName[:-3] + 'curve.txt', curve, fmt='%.6f  %.6f  %i')
			else:
				np.savetxt(curveFilePath + fileName[:-3] + 'curve.txt', curve, fmt='%.6f  %.6f')
			print('Curve file saved. ('+str(len(curve)) + ' points)')


In [ ]:
h5FileList = get_file_list(filePath, end='.h5')
h5FileList

In [ ]:


ii = 0

modeImage = []

for fileName in h5FileList:
	
	ii +=1
	
#	print(fileName[3:7])
	print(fileName)
	

		
	fileFullPath = os.path.join(filePath, fileName)
	
	spec, freq, velo = dispernet.readh5(fileFullPath)
	
	spec[np.isnan(spec)] = 0
	
	if np.sum(spec) == 0:
		continue

	threshold_set=threshold
	
	curve = []
	
	pSpec, period = dispernet.freq2Period(spec, freq, cutRate=0.1)
	pSpec = pSpec/np.max(pSpec)
	print(pSpec.shape)
	curveInPeriod = []

	for pdsInd in range(len(period)):
		pds = period[pdsInd]
		flag = 0
		for veloId, veloValue in enumerate(np.linspace(np.min(velo), np.max(velo), pSpec.shape[0])):
			
			if pSpec[veloId, pdsInd] > threshold_set and flag == 0:
				flag = 1
				begin_velo = veloValue
				
			elif flag == 1 and pSpec[veloId, pdsInd] <  threshold_set:
				mid_velo = begin_velo + (veloValue - begin_velo)/2
#				curveInPeriod.append([1/pds, begin_velo, 0])
				curveInPeriod.append([1/pds, mid_velo])
#				curveInPeriod.append([1/pds, veloValue,2])
				flag = 0
				
				
	curveInPeriod = np.array(curveInPeriod)
	if len(curve) == 0:
		curve = curveInPeriod
	else:
		if curve.shape[1] == 3:
			curveInPeriod = np.hstack((curveInPeriod, np.zeros([len(curveInPeriod),1])))
			
		curve = curve[curve[:,0] > 0.1]
		curve = np.vstack([curveInPeriod, curve])
		
	curve_save(curve, fileName , curveFilePath)
		